# Train a Simplicial 2-complex convolutional neural network (SCConv)


In this notebook, we will create and train a Simplicial 2-complex convolutional neural in the simplicial complex domain, as proposed in the paper by [Bunch et. al : Simplicial 2-Complex Convolutional Neural Networks (2020)](https://openreview.net/pdf?id=Sc8glB-k6e9).


We train the model to perform

The equations of one layer of this neural network are given by:

🟥 $\quad m_{y\rightarrow x}^{(0\rightarrow 0)} = ({\tilde{A}_{\uparrow,0}})_{xy} \cdot h_y^{t,(0)} \cdot \Theta^{t,(0\rightarrow0)}$

🟥 $\quad m^{(1\rightarrow0)}_{y\rightarrow x}  = (B_1)_{xy} \cdot h_y^{t,(0)} \cdot \Theta^{t,(1\rightarrow 0)}$

🟥 $\quad m^{(0 \rightarrow 1)}_{y \rightarrow x}  = (\tilde B_1)_{xy} \cdot h_y^{t,(0)} \cdot \Theta^{t,(0 \rightarrow1)}$

🟥 $\quad m^{(1\rightarrow1)}_{y\rightarrow x} = ({\tilde{A}_{\downarrow,1}} + {\tilde{A}_{\uparrow,1}})_{xy} \cdot h_y^{t,(1)} \cdot \Theta^{t,(1\rightarrow1)}$

🟥 $\quad m^{(2\rightarrow1)}_{y \rightarrow x}  = (B_2)_{xy} \cdot h_y^{t,(2)} \cdot \Theta^{t,(2 \rightarrow1)}$

🟥 $\quad m^{(1 \rightarrow 2)}_{y \rightarrow x}  = (\tilde B_2)_{xy} \cdot h_y^{t,(1)} \cdot \Theta^{t,(1 \rightarrow 2)}$

🟥 $\quad m^{(2 \rightarrow 2)}_{y \rightarrow x}  = ({\tilde{A}_{\downarrow,2}})\_{xy} \cdot h_y^{t,(2)} \cdot \Theta^{t,(2 \rightarrow 2)}$

🟧 $\quad m_x^{(0 \rightarrow 0)}  = \sum_{y \in \mathcal{L}_\uparrow(x)} m_{y \rightarrow x}^{(0 \rightarrow 0)}$

🟧 $\quad m_x^{(1 \rightarrow 0)}  = \sum_{y \in \mathcal{C}(x)} m_{y \rightarrow x}^{(1 \rightarrow 0)}$

🟧 $\quad m_x^{(0 \rightarrow 1)}  = \sum_{y \in \mathcal{B}(x)} m_{y \rightarrow x}^{(0 \rightarrow 1)}$

🟧 $\quad m_x^{(1 \rightarrow 1)}  = \sum_{y \in (\mathcal{L}_\uparrow(x) + \mathcal{L}_\downarrow(x))} m_{y \rightarrow x}^{(1 \rightarrow 1)}$

🟧 $\quad m_x^{(2 \rightarrow 1)} = \sum_{y \in \mathcal{C}(x)} m_{y \rightarrow x}^{(2 \rightarrow 1)}$

🟧 $\quad m_x^{(1 \rightarrow 2)}  = \sum_{y \in \mathcal{B}(x)} m_{y \rightarrow x}^{(1 \rightarrow 2)}$

🟧 $\quad m_x^{(2 \rightarrow 2)}  = \sum_{y \in \mathcal{L}_\downarrow(x)} m_{y \rightarrow x}^{(2 \rightarrow 2)}$

🟩 $\quad m_x^{(0)}  = m_x^{(1\rightarrow0)}+ m_x^{(0\rightarrow0)}$

🟩 $\quad m_x^{(1)}  = m_x^{(2\rightarrow1)}+ m_x^{(1\rightarrow1)}$

🟦 $\quad h^{t+1, (0)}_x  = \sigma(m_x^{(0)})$

🟦 $\quad h^{t+1, (1)}_x  = \sigma(m_x^{(1)})$

🟦 $\quad h^{t+1, (2)}_x  = \sigma(m_x^{(2)})$


Where the notations are defined in [Papillon et al : Architectures of Topological Deep Learning: A Survey of Topological Neural Networks (2023)](https://arxiv.org/abs/2304.10031).

In [88]:
import torch
import numpy as np

from toponetx import SimplicialComplex
import toponetx.datasets.graph as graph
import toponetx.datasets as datasets

from scipy.sparse import coo_matrix
from scipy.sparse import diags

from topomodelx.base.aggregation import Aggregation



# from topomodelx.nn.simplicial.scconv_layer import SCConvLayer

# Pre-processing

## Import dataset ##

The first step is to import the dataset, shrec 16, a benchmark dataset for 3D mesh classification.

In [3]:
shrec, _ = datasets.mesh.shrec_16(size="small")

shrec = {key: np.array(value) for key, value in shrec.items()}
x_0s = shrec["node_feat"]
x_1s = shrec["edge_feat"]
x_2s = shrec["face_feat"]

ys = shrec["label"]
simplexes = shrec["complexes"]

Loading shrec 16 small dataset...

done!


In [61]:
# l = np.unique(ys, return_counts=True)
# print(l)


(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]), array([1, 1, 4, 4, 5, 3, 2, 3, 3, 4, 4, 2, 3, 6, 1, 1, 3, 1, 4, 4, 7, 4,
       3, 1, 6, 1, 4, 5, 6, 4]))


In [4]:
i_complex = 0
print(
    f"The {i_complex}th simplicial complex has {x_0s[i_complex].shape[0]} nodes with features of dimension {x_0s[i_complex].shape[1]}."
)
print(
    f"The {i_complex}th simplicial complex has {x_1s[i_complex].shape[0]} edges with features of dimension {x_1s[i_complex].shape[1]}."
)
print(
    f"The {i_complex}th simplicial complex has {x_2s[i_complex].shape[0]} faces with features of dimension {x_2s[i_complex].shape[1]}."
)

The 0th simplicial complex has 252 nodes with features of dimension 6.
The 0th simplicial complex has 750 edges with features of dimension 10.
The 0th simplicial complex has 500 faces with features of dimension 7.


## Helper functions ##


In [27]:
def normalize_higher_order_adj(A_opt):
    """
    Args:
        A_opt is an opt that maps a j-cochain to a k-cochain.
        shape [num_of_k_simplices num_of_j_simplices]

    return:
         D^{-0.5}* (A_opt)* D^{-0.5}.
    """
    rowsum = np.array(np.abs(A_opt).sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.0
    r_mat_inv_sqrt = diags(r_inv_sqrt)
    A_opt_to = A_opt.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

    return coo_matrix(A_opt_to)


In [19]:
# incidence_1_list

In [43]:
adjacency_1 = simplexes[13].adjacency_matrix(rank=1, signed=False)
incidence_1 = simplexes[13].incidence_matrix(rank=1, signed=False)

# k = normalize_higher_order_adj(adjacency_1)
# print(k)

print(adjacency_1.todense().shape)
k = normalize_higher_order_adj(adjacency_1)
print(k.todense().shape)


(750, 750)
(750, 750)


# Define Neighbourhood Structures

We create the neigborood structures expected by SSConv. The SSConv layer expects the following neighbourhood structures:
* incidence_1 $B_1$
* incidence_1_norm $\tilde{B}_1$
* incidence_2 $B_2$
* incidence_2_norm $\tilde{B}_1$
* adjacency_up_0_norm $\tilde{A}_{\uparrow,0}$
* adjacency_up_1_norm $\tilde{A}_{\uparrow,1}$
* adjacency_down_1_norm $\tilde{A}_{\downarrow,1}$
* adjacency_down_2_norm $\tilde{A}_{\downarrow,2}$

In [87]:
def get_neighborhoods(simplexes):
    incidence_1_list = []
    incidence_2_list = []
    up_laplacian_1_list = []
    up_laplacian_2_list = []
    down_laplacian_1_list = []
    down_laplacian_2_list = []
    for simplex in simplexes:
        B1 = simplex.incidence_matrix(rank=1, signed=False)
        B2 = simplex.incidence_matrix(rank=2, signed=False)

        up_laplacian_1 = simplex.up_laplacian_matrix(rank=0)  #1
        up_laplacian_2 = simplex.up_laplacian_matrix(rank=1)  #2

        down_laplacian_1 = simplex.down_laplacian_matrix(rank=1)  #1
        down_laplacian_2 = simplex.down_laplacian_matrix(rank=2)  #2

        incidence_1 = torch.from_numpy(B1.todense()).to_sparse()
        incidence_2 = torch.from_numpy(B2.todense()).to_sparse()

        up_laplacian_1 = torch.from_numpy(up_laplacian_1.todense()).to_sparse()
        up_laplacian_2 = torch.from_numpy(up_laplacian_2.todense()).to_sparse()

        down_laplacian_1 = torch.from_numpy(down_laplacian_1.todense()).to_sparse()
        down_laplacian_2 = torch.from_numpy(down_laplacian_2.todense()).to_sparse()

        incidence_1_list.append(incidence_1)
        incidence_2_list.append(incidence_2)
        up_laplacian_1_list.append(up_laplacian_1)
        up_laplacian_2_list.append(up_laplacian_2)
        down_laplacian_1_list.append(down_laplacian_1)
        down_laplacian_2_list.append(down_laplacian_2)

    return incidence_1_list, incidence_2_list, up_laplacian_1_list, up_laplacian_2_list, down_laplacian_1_list, down_laplacian_2_list


incidence_1_list, incidence_2_list, up_laplacian_1_list, up_laplacian_2_list, down_laplacian_1_list, down_laplacian_2_list = get_neighborhoods(
    simplexes)

# Create the Neural Network

Using the SSConv class, we create a neural network with stacked layers.

In [91]:
class SCConv(torch.nn.Module):
    """Simplicial 2-Complex Convolutional Network Implementation for binary node classification.

    Parameters
    ---------
    node_channels : int
        Dimension of node (0-cells) features
    edge_channels : int
        Dimension of edge (1-cells) features
    face_channels : int
        Dimension of face (2-cells) features
    n_layers : int
        Number of message passing layers.
    n_classes : int
        Number of classes.
    update_func : str
        Activation function used in aggregation layers.

    """

    def __init__(self, node_channels, edge_channels, face_channels, n_classes, n_layers=2):
        super().__init__()
        self.node_channels = node_channels
        self.edge_channels = edge_channels
        self.face_channels = face_channels
        self.n_classes = n_classes


        layers = []
        for _ in range(n_layers):
            layers.append(
                SCConv(
                    node_channels=node_channels,
                    edge_channels=edge_channels,
                    face_channels=face_channels
                )
            )

        self.layers = torch.nn.ModuleList(layers)
        self.linear_x0 = torch.nn.Linear(node_channels, self.n_classes)
        self.linear_x1 = torch.nn.Linear(edge_channels, self.n_classes)
        self.linear_x2 = torch.nn.Linear(face_channels, self.n_classes)
        self.aggr = Aggregation(
            aggr_func="mean",
            update_func="sigmoid",
        )

    def forward(self,
                x_0,
                x_1,
                x_2,
                incidence_1,
                incidence_1_norm,
                incidence_2,
                incidence_2_norm,
                adjacency_up_0_norm,
                adjacency_up_1_norm,
                adjacency_down_1_norm,
                adjacency_down_2_norm):
        """Forward computation.

        Parameters
        ----------
        x_0: torch.Tensor, shape=[n_nodes, node_channels]
            Input features on the nodes of the simplicial complex.
        x_1: torch.Tensor, shape=[n_edges, edge_channels]
            Input features on the edges of the simplicial complex.
        x_2: torch.Tensor, shape=[n_faces, face_channels]
            Input features on the faces of the simplicial complex.
        incidence_1: torch.Tensor, shape=[n_faces, channels]
            incidence matrix of rank 1 :math:`B_1`.
        incidence_1_norm: torch.Tensor,
            normalized incidence matrix of rank 1 :math:`B^{~}_1`.
        incidence_2: torch.Tensor,
             incidence matrix of rank 2 :math:`B_2`.
        incidence_2_norm: torch.Tensor,
            normalized incidence matrix of rank 2 :math:`B^{~}_2`.
        adjacency_up_0_norm: torch.Tensor,
            normalized upper adjacency matrix of rank 0.
        adjacency_up_1_norm: torch.Tensor,
            normalized upper adjacency matrix of rank 1.
        adjacency_down_1_norm: torch.Tensor,
            normalized down adjacency matrix of rank 1.
        adjacency_down_2_norm: torch.Tensor,
            normalized down adjacency matrix of rank 2.

        Returns
        --------
        _ : tensor, shape = [1]
            Label assigned to whole complex.

        """

        for i in range(self.n_layers):
            x_0, x_1, x_2 = self.layers[i](x_0,
                                           x_1,
                                           x_2,
                                           incidence_1,
                                           incidence_1_norm,
                                           incidence_2,
                                           incidence_2_norm,
                                           adjacency_up_0_norm,
                                           adjacency_up_1_norm,
                                           adjacency_down_1_norm,
                                           adjacency_down_2_norm)

        x_0 = self.linear_x0(x_0)
        x_1 = self.linear_x1(x_1)
        x_2 = self.linear_x2(x_2)

        node_mean = torch.nanmean(x_0, dim=0)
        node_mean[torch.isnan(node_mean)] = 0

        edge_mean = torch.nanmean(x_1, dim=0)
        edge_mean[torch.isnan(edge_mean)] = 0

        face_mean = torch.nanmean(x_2, dim=0)
        face_mean[torch.isnan(face_mean)] = 0

        return torch.flatten(node_mean) + torch.flatten(edge_mean) + torch.flatten(face_mean)

# Train the Neural Network

We specify the model with our pre-made neighborhood structures and specify an optimizer.

The following cell performs the training, looping over the network for a low number of epochs.